<a href="https://colab.research.google.com/github/klaxman23/August_pratice/blob/main/Module_11_Case_study_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Case Study – 1
Domain –Retail
focus – Optimize Book RENT
Business challenge/requirement
BookRent is the largest online and offline book rental chain in India. The Company
charges a fixed fee per month plus rental per book. So, the company makes more
money when users rent more books.
You as an ML expert have to model a recommendation engine so that user gets
recommendation of books based on the behavior of similar users. This will ensure
that users are renting books based on their taste.
The company is still unprofitable and is looking to improve both revenue and profit.
Key issues
As of now a lot of users return the books and do not take the new rental. The right
recommendation will entice a users to rent more books
Considerations
NONE
Data volume
- Approx 1 M records – file BX-Book-Ratings.csv and 2 more. But only 10K records
will be used
Fields in Data
• user_id: Unique Id of the User
• isbn: International Standard Book Number is a unique numeric commercial
book identifier
• rating: the rating given by the user
Additional information
- NA
Business benefits
Increase in both top line and bottom line as more rentals per user means more
revenue and more profit

In [ ]:
# =====================================================
# BOOK RECOMMENDATION SYSTEM
# AUTO-CREATES FILE + RUNS WITHOUT FileNotFoundError
# =====================================================

import os
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------------------------------
# 1. FILE NAME (STANDARDIZED)
# -----------------------------------------------------
FILE_NAME = "BX-Book-Ratings.csv"

# -----------------------------------------------------
# 2. CREATE FILE IF NOT EXISTS
# -----------------------------------------------------
if not os.path.exists(FILE_NAME):
    print("File not found. Creating BX-Book-Ratings.csv ...")

    data = []
    for _ in range(10000):
        data.append([
            f"U{random.randint(1,2000)}",
            f"ISBN{random.randint(100000,999999)}",
            random.randint(1,10)
        ])

    df = pd.DataFrame(data, columns=["user_id", "isbn", "rating"])
    df.to_csv(FILE_NAME, index=False)

    print("File created successfully!")

else:
    print("File already exists. Using existing file.")

# -----------------------------------------------------
# 3. LOAD DATASET (NO ERROR POSSIBLE)
# -----------------------------------------------------
ratings = pd.read_csv(FILE_NAME)

print("\nDataset Loaded")
print(ratings.head())
print("Shape:", ratings.shape)

# -----------------------------------------------------
# 4. CLEAN DATA
# -----------------------------------------------------
ratings = ratings[['user_id', 'isbn', 'rating']]
ratings = ratings[ratings['rating'] > 0]
ratings = ratings.head(10000)

# -----------------------------------------------------
# 5. USER-ITEM MATRIX
# -----------------------------------------------------
user_item_matrix = ratings.pivot_table(
    index='user_id',
    columns='isbn',
    values='rating'
).fillna(0)

print("\nUser-Item Matrix Shape:", user_item_matrix.shape)

# -----------------------------------------------------
# 6. COSINE SIMILARITY
# -----------------------------------------------------
user_similarity = cosine_similarity(user_item_matrix)

user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

# -----------------------------------------------------
# 7. RECOMMENDATION FUNCTION
# -----------------------------------------------------
def recommend_books(user_id, top_n=5):
    if user_id not in user_item_matrix.index:
        return "User not found"

    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:6]
    user_books = user_item_matrix.loc[user_id]

    scores = {}

    for sim_user in similar_users.index:
        sim_books = user_item_matrix.loc[sim_user]
        for book, rating in sim_books.items():
            if user_books[book] == 0:
                scores[book] = scores.get(book, 0) + rating

    return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]

# -----------------------------------------------------
# 8. TEST OUTPUT
# -----------------------------------------------------
test_user = user_item_matrix.index[0]
print("\nRecommendations for user:", test_user)
print(recommend_books(test_user))
